# Ranking the drawdowns of Index constituents

It is useful to keep a tab on drawdowns of index constituents to get a feel for how the index is diverging/converging from individual stock performance. For example, 7 stocks account for 50% of the NIFTY 50's performance and end up masking the performance of the other 43. If the rest are crashing, how long will the best hold up?

StockViz has an automated report on drawdowns of NIFTY 100, MIDCAP 150 and SMLCAP 100 constituents that gets generated everyday [here](https://stockviz.github.io/reports/drawdown/drawdowns.html). The [underlying code](https://github.com/stockviz/reports/tree/master/drawdown) is an R-markdown script who's output gets pushed to github. In this notebook, by setting the *indexName* variable to any NSE/NIFTY index that we have constituents for, you can generate the same report for yourself on [pluto](http://pluto.studio).

In [ ]:
library(tidyverse)
library(ggthemes)
library(reshape2)
library(odbc)
library(RPostgres)
library(plutoR)
library(quantmod)
library(lubridate)
library(ggrepel)
library(PerformanceAnalytics)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

#initialize
indices<-Indices()
equitiesIndiaNse <- EquitiesIndiaNse()

indexName <- "NIFTY 50"
endDate <- Sys.Date()
startDate <- endDate - 365 

In [ ]:
maxDt <- (indices$NseConstituents() %>%
          filter(NAME == indexName) %>%
          summarize(MAX_TS = max(TIME_STAMP)) %>% 
          collect())$MAX_TS[1]

constituents <- indices$NseConstituents() %>%
    filter(TIME_STAMP == maxDt & NAME == "NIFTY 50") %>%
    collect() %>%
    as.data.frame()

### plot the weights to get a feel for individual contributions

In [ ]:
toPlot <- constituents

ggplot(toPlot, aes(x=reorder(SYMBOL, -CAP_WEIGHT), y=CAP_WEIGHT)) +
    theme_economist() +
    theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
    geom_bar(stat="identity", position=position_dodge()) +
    geom_text_repel(aes(label=round(CAP_WEIGHT, 2)), position = position_dodge(0.9)) +
    labs(x='', y='free-float cap weight (%)', color='', fill='', title=sprintf("%s Composition", indexName)) +
    annotate("text", x=1, y=0, label = "@StockViz", 
             hjust=-1.1, vjust=-1.1, col="white", cex=6, fontface = "bold", alpha = 0.8)

#### modify the drawdown report *genTables* function to work on pluto

In [ ]:
# https://github.com/shyams80/plutons/blob/master/docs-R/EquitiesIndiaNse.ipynb

endDate <- (equitiesIndiaNse$DailyReturns() %>%
            filter(TIME_STAMP <= endDate) %>%
            summarize(MAX_DT = max(TIME_STAMP)) %>%
            collect())$MAX_DT[[1]]

numCalDays <- as.integer(endDate-startDate)
ongoingDf <- data.frame(SYMBOL = "", FROM="", DEPTH = 0.0)
recoveringDf <- data.frame(SYMBOL = "", FROM="", TROUGH="", DEPTH = 0.0, BOUNCE = 0.0)
for(sym in constituents$SYMBOL){
    retSeries <- equitiesIndiaNse$DailyReturns() %>%
        filter(SYMBOL == sym & TIME_STAMP >= startDate & TIME_STAMP <= endDate) %>%
        collect()
    
    if(nrow(retSeries) < 0.5*numCalDays) next
    if(last(retSeries$TIME_STAMP) != endDate) next

    retXts <- xts(retSeries$VALUE, as.Date(retSeries$TIME_STAMP))
    tdd <- table.Drawdowns(retXts, 10)

    notRec <- tdd[is.na(tdd$Recovery),]
    if(nrow(notRec) > 0) {
        ongoingDf <- rbind(ongoingDf, c(toString(sym), toString(notRec$From[1]), notRec$Depth[1]))
    }

    recovering <- notRec[!is.na(notRec$Trough) & notRec$Trough < endDate,]
    if(nrow(recovering) > 0) {
        deepest <- recovering[order(recovering$Depth),][1,]
        fromDt <- as.Date(deepest$Trough[1])
        bounce <- Return.cumulative(retXts[sprintf("%s/%s", fromDt, endDate)])

        if (bounce > 0) {
            recoveringDf <- rbind(recoveringDf, c(toString(sym), 
                                                  toString(deepest$From[1]), toString(fromDt), 
                                                  deepest$Depth[1], bounce))
        }
    }
}

ongoingDf <- ongoingDf[-1,]
recoveringDf <- recoveringDf[-1,]

ongoingDf$DEPTH <- 100*as.numeric(ongoingDf$DEPTH)
recoveringDf$DEPTH <- 100*as.numeric(recoveringDf$DEPTH)
recoveringDf$BOUNCE <- round(100*as.numeric(recoveringDf$BOUNCE), 2)

ongoingDf <- ongoingDf[order(ongoingDf$DEPTH),]
recoveringDf <- recoveringDf[order(recoveringDf$BOUNCE, decreasing = T),]

### Ongoing Drawdowns

In [ ]:
print(ongoingDf)

### Recovering

In [ ]:
print(recoveringDf)

### Cap-weight + deepest drawdown

In [ ]:
toPlot <- merge(constituents[, c('SYMBOL', 'CAP_WEIGHT')], ongoingDf[, c('SYMBOL', 'DEPTH')], by='SYMBOL')
toPlot <- toPlot[order(toPlot$CAP_WEIGHT, decreasing = T),]
print(toPlot)